# Idiomatic Pandas

In [1]:
import pandas as pd
import numpy as np

## Use case 1: modify dataframe content

Create gender data.

In [2]:
# produce random indexes
np.random.seed(7)
rand_indices = np.random.randint(0, 2, 1000)
# generate random list of 'M' and 'F'
gender = ['M', 'F']
gender_list = [ [gender[k]] for k in rand_indices]
# create DataFrame
df = pd.DataFrame(gender_list, columns=['Gender'])
df.head()

,Gender
0,F
1,M
2,F
3,M
4,F


### Goal: Replace Male ('M') by 0 and Female ('F') by 1

Time your code using 
```python
%timeit one_line_statement
```
Or for an entire cell
```python
%%timeit
 line_statement...
```

### Solution 1 (It works but not great)

In [3]:
df = pd.DataFrame(gender_list, columns=['Gender'])

In [4]:
%%timeit
# SHOULD give SettingWithCopyWarning (double indexing, get Gender column then boolean)
# Still performs assignment correctly
df.Gender[df.Gender == 'M'] = 0
df.Gender[df.Gender == 'F'] = 1

3.62 ms ± 309 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Solution 2 (Don't do it!)

In [25]:
df = pd.DataFrame(gender_list, columns=['Gender'])

In [26]:
%%timeit
# equivalent to solution 1, SHOULD give SettingWithCopyWarning but works
df.loc[:,'Gender'][df.Gender == 'M'] = 0
df.loc[:,'Gender'][df.Gender == 'F'] = 1

3.31 ms ± 130 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Solution 3 (Don't do even think about it!)

In [27]:
df = pd.DataFrame(gender_list, columns=['Gender'])

In [28]:
%%timeit
# The following statements produces SettingWithCopyWarning AND doesn't do the assigment
# typical chain indexing, meaning indexing happens twice. It makes an assignment on a copy in this case!
df[df.Gender == 'M'].loc[:,'Gender'] = 0
df[df.Gender == 'F'].loc[:,'Gender'] = 1

/home/ceddie/anaconda3/envs/insight/lib/python3.6/site-packages/pandas/core/indexing.py:630: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


33.7 ms ± 2.42 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Solution 4 (.apply does well here but it shouldn't TBD!)

In [45]:
df = pd.DataFrame(gender_list, columns=['Gender'])

In [46]:
# works, no warning but not the fastest (avoid using apply because it is slow in general)
df['Gender'] = df['Gender'].apply(lambda x: 0 if x == 'M' else 1)

In [47]:
# works, no warning but not the fastest (avoid using apply because it is slow in general)
%timeit df['Gender'] = df['Gender'].apply(lambda x: 0 if x == 'M' else 1)

543 µs ± 29.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Solution 5 (Correct and Preferred Solution)

In [5]:
df = pd.DataFrame(gender_list, columns=['Gender'])

In [6]:
%timeit df.loc[df.Gender == 'M', 'Gender'] = 0

1.49 ms ± 43.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [54]:
# CORRECT SOLUTION single indexing. Produces correct ouput and fastest solution
df.loc[df.Gender == 'M', 'Gender'] = 0
df.loc[df.Gender == 'F', 'Gender'] = 1

# Case 2: Add new column using .map() and dictionaries

Create a dataframe based on real data

In [7]:
county_index  = ['Adams', 'Allegheny', 'Armstrong' ,  'Beaver' , 'Bedford']
cols = ['state', 'total', 'Obama', 'Romney', 'winner', 'voters']

state   = ['PA']*5
total  = [  41973  ,  614671   ,  28322  ,  80015  ,  21444]
Obama = [ 35.4823 ,  56.6402  ,  30.697 , 46.0326 , 22.0575]
Romney = [ 63.112 ,  42.1858 ,  67.9013 , 52.6376 , 76.9866]
winner =  ['Romney'  ,   'Obama'  ,  'Romney' ,  'Romney' ,  'Romney']
voters =  [ 61156 ,   924351  ,   42147 ,  115157  ,  32189]

election = pd.DataFrame(np.array([state, total, Obama, Romney, winner, voters]).T, index=county_index, columns=cols)
election

,state,total,Obama,Romney,winner,voters
Adams,PA,41973,35.4823,63.112,Romney,61156
Allegheny,PA,614671,56.6402,42.1858,Obama,924351
Armstrong,PA,28322,30.697,67.9013,Romney,42147
Beaver,PA,80015,46.0326,52.6376,Romney,115157
Bedford,PA,21444,22.0575,76.9866,Romney,32189


### Add a new column `color` that shows red or blue according to `winner`

In [8]:
red_vs_blue = {'Obama':'blue', 'Romney':'red'}

In [9]:
election['color'] = election['winner'].map(red_vs_blue)
election

,state,total,Obama,Romney,winner,voters,color
Adams,PA,41973,35.4823,63.112,Romney,61156,red
Allegheny,PA,614671,56.6402,42.1858,Obama,924351,blue
Armstrong,PA,28322,30.697,67.9013,Romney,42147,red
Beaver,PA,80015,46.0326,52.6376,Romney,115157,red
Bedford,PA,21444,22.0575,76.9866,Romney,32189,red


# Case 3: Input missing values

Add another index (row) and leave total column at zero (i.e. missing)

In [10]:
election.loc['wonderland', :] = ['PA', 0, 33.5, 66.5, 'Romney', 45000, 'red']
# make sure total is numeric
election.total = pd.to_numeric(election.total)
election

,state,total,Obama,Romney,winner,voters,color
Adams,PA,41973,35.4823,63.112,Romney,61156,red
Allegheny,PA,614671,56.6402,42.1858,Obama,924351,blue
Armstrong,PA,28322,30.697,67.9013,Romney,42147,red
Beaver,PA,80015,46.0326,52.6376,Romney,115157,red
Bedford,PA,21444,22.0575,76.9866,Romney,32189,red
wonderland,PA,0,33.5,66.5,Romney,45000,red


### Again using .map() and dictionaries

In [ ]:
# create mask
mask_column = (election['total'] == 0)

# create a serie with index as region and values as mean for that region
s_group = election[~mask_column].groupby('winner')['total'].mean()

In [ ]:
# to_dict() is optional, map() accepts series!
election.loc[mask_column, 'total'] = election.loc[mask_column, 'winner'].map(s_group.to_dict())
election

### Again but using transform
PS: missing value must be Nan for transform to work!

In [ ]:
# set last record of total to 0 again
election.loc['wonderland', :] = ['PA', 0, 33.5, 66.5, 'Romney', 45000, 'red']
# replace missing value with Nan value
election.loc[election.total == 0, 'total'] = np.nan
election

In [ ]:
# Write a function that imputes mean
def impute_mean(series):
    return series.fillna(series.mean())

In [ ]:
# replace Nan values with mean for each sub serie created by groupby
election.total = election.groupby('winner')['total'].transform(impute_mean)
election

### Again using reindex()

In [ ]:
# set last record of total to 0 again
election.loc['wonderland', :] = ['PA', 0, 33.5, 66.5, 'Romney', 45000, 'red']

In [ ]:
# create mask
mask_column = (election['total'] == 0)

# create a serie with index as region and values as mean for that region
s_group = election[~mask_column].groupby('winner')['total'].mean()

# index by labels hence 
# create a serie aligned with df[to_groupby].values order, it marks not found label as Nan hence
# resulting serie has the same number of elements as in Dataframe
#     mean_col = s_group[df[to_groupby].values].values

# idiomatic way to do the same operation (above method is depreciated)
mean_col = s_group.reindex(election['winner'].values).values

# assign column equal to value with their mean determined by region
# use mask_column on df and ndarray (it has to be an ndarray as opposed to a serie because
# mask_column, the boolean indexer, does not have the same index as mean_col would have if it were left as a serie
# (winner index vs county index)
election.loc[mask_column, 'total'] = mean_col[mask_column]
election